In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_staging_delivery_order")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCCSDDL52"
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_delivery_order(
    hash_id BIGINT, 
    calday DATE, 
    do_number STRING, 
    do_item STRING, 
    creation_date DATE, 
    gi_date DATE, 
    act_gi_dte DATE, 
    deleted_record STRING, 
    product_id STRING, 
    deliver_qty_sale DOUBLE,
    sales_unit STRING, 
    deliver_qty_base DOUBLE,
    base_unit STRING, 
    dc_id STRING, 
    sto_number STRING, 
    sto_item STRING, 
    sts_pick STRING, 
    osts_pick STRING, 
    tgoodmv_st STRING, 
    pick_conf STRING, 
    goodsmv_st STRING, 
    recordmode STRING, 
    unload_pt STRING, 
    imounl_pt STRING, 
    sales_dist STRING, 
    bill_block STRING, 
    incoterms STRING, 
    incoterms2 STRING, 
    imoincotm2 STRING, 
    cust_group STRING, 
    del_type STRING, 
    ship_date STRING, 
    creditor STRING, 
    del_block STRING, 
    load_pt STRING, 
    route STRING, 
    doc_categ STRING, 
    ch_on STRING, 
    rt_promo STRING, 
    batch STRING, 
    eanupc STRING, 
    createdby STRING, 
    crea_time STRING, 
    bilblk_dl STRING, 
    bus_area STRING, 
    pick_indc STRING, 
    cust_grp1 STRING, 
    cust_grp2 STRING, 
    cust_grp3 STRING, 
    cust_grp4 STRING, 
    cust_grp5 STRING, 
    consu_flag STRING, 
    whse_num STRING, 
    stor_loc STRING, 
    plant STRING, 
    strge_bin STRING, 
    strge_type STRING, 
    itm_type STRING, 
    forwagent STRING, 
    item_categ STRING, 
    per_resp STRING, 
    stat_date STRING, 
    prvdoc_ctg STRING, 
    imodoccat STRING, 
    wbs_elemt STRING, 
    del_wa_dh STRING, 
    calmonth STRING, 
    calquarter STRING, 
    calyear STRING, 
    fiscvarnt STRING, 
    fiscper STRING, 
    fiscyear STRING, 
    upd_date STRING, 
    no_del_it DOUBLE,
    denomintr DOUBLE,
    numerator DOUBLE,
    shp_pr_tmf DOUBLE,
    shp_pr_tmv DOUBLE
    )
    TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_{target_table}  
    AS
        WITH temp AS (
            SELECT
                reqtsn,
                datapakid,
                record,
                calday,
                do_number,
                do_item,
                creation_date,
                gi_date,
                act_gi_dte,
                deleted_record,
                product_id,
                deliver_qty_sale,
                sales_unit,
                deliver_qty_base,
                base_unit,
                dc_id,
                sto_number,
                sto_item,
                sts_pick,
                osts_pick,
                tgoodmv_st,
                pick_conf,
                goodsmv_st,
                recordmode,
                unload_pt,
                imounl_pt,
                sales_dist,
                bill_block,
                incoterms,
                incoterms2,
                imoincotm2,
                cust_group,
                del_type,
                ship_date,
                creditor,
                del_block,
                load_pt,
                route,
                doc_categ,
                ch_on,
                rt_promo,
                batch,
                eanupc,
                createdby,
                crea_time,
                bilblk_dl,
                bus_area,
                pick_indc,
                cust_grp1,
                cust_grp2,
                cust_grp3,
                cust_grp4,
                cust_grp5,
                consu_flag,
                whse_num,
                stor_loc,
                plant,
                strge_bin,
                strge_type,
                itm_type,
                forwagent,
                item_categ,
                per_resp,
                stat_date,
                prvdoc_ctg,
                imodoccat,
                wbs_elemt,
                del_wa_dh,
                calmonth,
                calquarter,
                calyear,
                fiscvarnt,
                fiscper,
                fiscyear,
                upd_date,
                no_del_it,
                denomintr,
                numerator,
                shp_pr_tmf,
                shp_pr_tmv
            FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccsddl52
            WHERE calday IN ({CALDAY_IN_STR}) 
            QUALIFY ROW_NUMBER() OVER (PARTITION BY reqtsn, datapakid, record ORDER BY proc_date DESC) = 1 
        ),
        main as (
            SELECT 
                CAST(farm_fingerprint(CONCAT(
                    IFNULL(CAST(do_number AS STRING), ""),
                    IFNULL(CAST(do_item AS STRING), "")
                )) AS BIGINT) hash_id,
                * EXCEPT(reqtsn, datapakid, record)
            FROM temp
            QUALIFY ROW_NUMBER() OVER (PARTITION BY hash_id ORDER BY reqtsn DESC, datapakid DESC, record DESC) = 1
        )
        SELECT *
        FROM main;
    """)

else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != '':
    calday_query = f"""
        SELECT 
            CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
        FROM temp_{target_table}
        """
    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_delivery_order 
        WHERE hash_id IN (SELECT hash_id FROM temp_{target_table});
""")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_delivery_order").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_delivery_order ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_delivery_order can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"

In [0]:
dbutils.notebook.exit("Success")